# k-Abelian Complexity of Tribonacci sequence from 1 to 5

## Initial Numeration System Setup

First we define the subsitution, its numeration system and its fixpoint
sequence.

In [2]:
%%python
from licofage.kit import *
import os
setparams(True, True, os.environ["WALNUT_HOME"])

s = subst('01/02/0')
ns = address(s, "tri")
ns.gen_ns()
ns.gen_word_automaton()

>>> Address automaton for subst(0->01, 1->02, 2->0)
  DT polynomial: X^3-X^2-X-1 (n=3)
  θ=1.839286755214161
  DT automaton: 3 states, 3 final states, 5 transitions.

>>> address(subst(0->01, 1->02, 2->0), 'tri') is generating it's NS
  Numeration system automaton: 3 states, 3 final states, 5 transitions.
  Writing /var/folders/l8/fphf93t95171xj2_rf0knxtc0000gn/T/tmp.wBJ3QLoyq3/Walnut/Custom Bases/msd_tri.txt in format Walnut
  +address(subst(0->01, 1->02, 2->0), 'tri')+address(subst(0->01, 1->02, 2->0), 'tri')-address(subst(0->01, 1->02, 2->0), 'tri') is generating it's dfa under name msd_tri_addition
  Combination polynomial is X^3-X^2-X-1
  θ=1.839286755214161
  Combination automaton: 27 states, 27 final states, 125 transitions.
  (P,u) vector bound: (0.0, 0.0, 0.0) + (5.770539317125755, 8.360221985276333, 13.123391017013361)*C
  Fast poly bound: C in [(-1, -1, -1), (2, 2, 2)] => (34.62323590275453, 50.161331911657996, 78.74034610208017)
Expanding DFA...done.
  Flattened automaton :

Then we setup a factor comparison predicate in Walnut and a first factor
occurence predicate.

In [3]:
def cut "?msd_tri i<=u & j<=v & u+j=v+i & u<n+i & v<n+j":
def feq_tri "?msd_tri ~(Eu,v $cut(i,j,n,u,v) & Tri[u]!=Tri[v])":
eval comp_tri n "?msd_tri Aj $feq_tri(i,j,n) => i<=j":

i<=u:12 states - 5ms
 j<=v:12 states - 1ms
  (i<=u&j<=v):144 states - 3ms
   (u+j)=(v+i):1087 states - 194ms
    ((i<=u&j<=v)&(u+j)=(v+i)):1096 states - 12ms
     u<(n+i):228 states - 9ms
      (((i<=u&j<=v)&(u+j)=(v+i))&u<(n+i)):11722 states - 264ms
       v<(n+j):228 states - 10ms
        ((((i<=u&j<=v)&(u+j)=(v+i))&u<(n+i))&v<(n+j)):11722 states - 191ms
Total computation time: 698ms.

Tri[u]!=Tri[v]:9 states - 0ms
 (cut(i,j,n,u,v))&Tri[u]!=Tri[v]):11722 states - 146ms
  (E u , v (cut(i,j,n,u,v))&Tri[u]!=Tri[v])):63 states - 10945ms
   ~(E u , v (cut(i,j,n,u,v))&Tri[u]!=Tri[v])):26 states - 2ms
Total computation time: 11229ms.

i<=j:12 states - 0ms
 (feq_tri(i,j,n))=>i<=j):66 states - 3ms
  (A j (feq_tri(i,j,n))=>i<=j)):22 states - 3ms
Total computation time: 6ms.


From there we can define the boundary condition `bordercond`, as
explained in Fici-Puzynina-2023 section 8.1.

In [4]:
def bordercond "?msd_tri (k<=n => $feq_tri(i,j,k-1)) & (n<k => $feq_tri(i,j,n))":

k<=n:12 states - 0ms
computed ~:1 states - 6ms
computed ~:2 states - 0ms
 (k<=n=>feq_tri(i,j,(k-1)))):222 states - 22ms
  n<k:12 states - 1ms
   (n<k=>feq_tri(i,j,n))):222 states - 5ms
    ((k<=n=>feq_tri(i,j,(k-1))))&(n<k=>feq_tri(i,j,n)))):148 states - 2ms
Total computation time: 38ms.


## 1-abelian complexity

Define the 1-block map of tri and construct the conversion predicate
between both numeration systems.

In [5]:
%%python
s1 = s
ns1 = ns

### Compute Parikh vectors

First we compute the Parikh vectors for the prefixes of Tri.

In [6]:
%%python
for (i,a) in enumerate(ns1.alpha):
    w = {'_': 0}
    w[a] = 1
    parikh = address(s1, ns1.ns, **w)
    (parikh - ns1).gen_dfa(f"trib1p{i}")

>>> Address automaton for subst(0->01, 1->02, 2->0)
  DT polynomial: X^3-X^2-X-1 (n=3)
  θ=1.839286755214161
  DT automaton: 3 states, 3 final states, 5 transitions.

>>> +address(subst(0->01, 1->02, 2->0), 'tri', '_': 0, '0': 1)-address(subst(0->01, 1->02, 2->0), 'tri') is generating it's dfa under name trib1p0
  Combination polynomial is X^3-X^2-X-1
  θ=1.839286755214161
  Combination automaton: 9 states, 9 final states, 25 transitions.
  (P,u) vector bound: (0.0, 0.0, 0.0) + (5.770539317125755, 8.360221985276333, 13.123391017013361)*C
  Fast poly bound: C in [(-1, -1, -1), (0, 0, 1)] => (17.311617951377265, 25.080665955828998, 39.370173051040084)
Expanding DFA...done.
  Flattened automaton : 310 states, 7 final states, 473 transitions.
  Trimmed flattened automaton : 10 states, 7 final states, 16 transitions.
  Real bound is [1, 1, 2] (vs [17.311617951377265, 25.080665955828998, 39.370173051040084]).
  After minimization: 7 states, 4 final states, 11 transitions.
  Writing /var/fold

### Compute the complexity function

Use Walnut to generate a linear representation for the 1-abelian
complexity.

In [7]:
def fac0 "?msd_tri Ex,y $trib1p0(i,x) & $trib1p0(i+n,y) & z+x=y":
def fac1 "?msd_tri Ex,y $trib1p1(i,x) & $trib1p1(i+n,y) & z+x=y":
def fac2 "?msd_tri Ex,y $trib1p2(i,x) & $trib1p2(i+n,y) & z+x=y":

(trib1p0(i,x))&trib1p0((i+n),y))):394 states - 1ms
 (z+x)=y:149 states - 1ms
  ((trib1p0(i,x))&trib1p0((i+n),y)))&(z+x)=y):712 states - 6ms
   (E x , y ((trib1p0(i,x))&trib1p0((i+n),y)))&(z+x)=y)):239 states - 1ms
Total computation time: 12ms.

(trib1p1(i,x))&trib1p1((i+n),y))):588 states - 4ms
 (z+x)=y:149 states - 1ms
  ((trib1p1(i,x))&trib1p1((i+n),y)))&(z+x)=y):1286 states - 13ms
   (E x , y ((trib1p1(i,x))&trib1p1((i+n),y)))&(z+x)=y)):283 states - 3ms
Total computation time: 26ms.

(trib1p2(i,x))&trib1p2((i+n),y))):973 states - 2ms
 (z+x)=y:149 states - 1ms
  ((trib1p2(i,x))&trib1p2((i+n),y)))&(z+x)=y):2672 states - 55ms
   (E x , y ((trib1p2(i,x))&trib1p2((i+n),y)))&(z+x)=y)):406 states - 6ms
Total computation time: 69ms.


In [8]:
def min0 "?msd_tri Ei $fac0(i,n,x) & Aj,y $fac0(j,n,y) => y>=x":
def min1 "?msd_tri Ei $fac1(i,n,x) & Aj,y $fac1(j,n,y) => y>=x":
def min2 "?msd_tri Ei $fac2(i,n,x) & Aj,y $fac2(j,n,y) => y>=x":

y>=x:12 states - 0ms
 (fac0(j,n,y))=>y>=x):1266 states - 54ms
  (A j , y (fac0(j,n,y))=>y>=x)):81 states - 56ms
   (fac0(i,n,x))&(A j , y (fac0(j,n,y))=>y>=x))):1112 states - 3ms
    (E i (fac0(i,n,x))&(A j , y (fac0(j,n,y))=>y>=x)))):87 states - 3ms
Total computation time: 118ms.

y>=x:12 states - 0ms
 (fac1(j,n,y))=>y>=x):1690 states - 62ms
  (A j , y (fac1(j,n,y))=>y>=x)):86 states - 64ms
   (fac1(i,n,x))&(A j , y (fac1(j,n,y))=>y>=x))):1174 states - 2ms
    (E i (fac1(i,n,x))&(A j , y (fac1(j,n,y))=>y>=x)))):98 states - 2ms
Total computation time: 133ms.

y>=x:12 states - 0ms
 (fac2(j,n,y))=>y>=x):2515 states - 72ms
  (A j , y (fac2(j,n,y))=>y>=x)):97 states - 80ms
   (fac2(i,n,x))&(A j , y (fac2(j,n,y))=>y>=x))):1326 states - 3ms
    (E i (fac2(i,n,x))&(A j , y (fac2(j,n,y))=>y>=x)))):113 states - 3ms
Total computation time: 160ms.


In [9]:
def diff0 "?msd_tri Ex,y $min0(n,x) & $fac0(i,n,y) & z+x=y":
def diff1 "?msd_tri Ex,y $min1(n,x) & $fac1(i,n,y) & z+x=y":
def diff2 "?msd_tri Ex,y $min2(n,x) & $fac2(i,n,y) & z+x=y":

(min0(n,x))&fac0(i,n,y))):1432 states - 6ms
 (z+x)=y:149 states - 1ms
  ((min0(n,x))&fac0(i,n,y)))&(z+x)=y):1405 states - 6ms
   (E x , y ((min0(n,x))&fac0(i,n,y)))&(z+x)=y)):1083 states - 6ms
Total computation time: 21ms.

(min1(n,x))&fac1(i,n,y))):1615 states - 6ms
 (z+x)=y:149 states - 1ms
  ((min1(n,x))&fac1(i,n,y)))&(z+x)=y):1611 states - 7ms
   (E x , y ((min1(n,x))&fac1(i,n,y)))&(z+x)=y)):1088 states - 6ms
Total computation time: 21ms.

(min2(n,x))&fac2(i,n,y))):1931 states - 8ms
 (z+x)=y:149 states - 2ms
  ((min2(n,x))&fac2(i,n,y)))&(z+x)=y):1920 states - 10ms
   (E x , y ((min2(n,x))&fac2(i,n,y)))&(z+x)=y)):1089 states - 7ms
Total computation time: 28ms.


In [10]:
def abeq_tri1 "?msd_tri (Ez $diff0(i,n,z) &  $diff0(j,n,z)) & (Ez $diff1(i,n,z) &  $diff1(j,n,z)) & (Ez $diff2(i,n,z) &  $diff2(j,n,z))":

(diff0(i,n,z))&diff0(j,n,z))):16995 states - 103ms
 (E z (diff0(i,n,z))&diff0(j,n,z)))):2613 states - 54ms
  (diff1(i,n,z))&diff1(j,n,z))):17010 states - 99ms
   (E z (diff1(i,n,z))&diff1(j,n,z)))):2613 states - 52ms
    ((E z (diff0(i,n,z))&diff0(j,n,z))))&(E z (diff1(i,n,z))&diff1(j,n,z))))):2613 states - 8ms
     (diff2(i,n,z))&diff2(j,n,z))):17015 states - 87ms
      (E z (diff2(i,n,z))&diff2(j,n,z)))):2613 states - 51ms
       (((E z (diff0(i,n,z))&diff0(j,n,z))))&(E z (diff1(i,n,z))&diff1(j,n,z)))))&(E z (diff2(i,n,z))&diff2(j,n,z))))):2613 states - 8ms
Total computation time: 484ms.


In [11]:
eval comp_tri1 n "?msd_tri Aj $abeq_tri1(i,j,n) => i<=j":

i<=j:12 states - 0ms
 (abeq_tri1(i,j,n))=>i<=j):2741 states - 39ms
  (A j (abeq_tri1(i,j,n))=>i<=j)):185 states - 48ms
Total computation time: 96ms.


### Apply the semigroup trick

In [12]:
%SGT comp_tri1 msd_tri Comp_tri1

Reading Result/comp_tri1.mpl
Rank: 63
78 states
values: [0, 1, 3, 4, 5, 6, 7]
Writing Word Automata Library/Comp_tri1.txt
Done.

## 2-abelian complexity

Define the 2-block map of tri and construct the conversion predicate
between both numeration systems.

In [13]:
%%python
s2 = block(s, 2)
ns2 = address(s2, "trib2")
ns2.gen_ns()
(ns-ns2).gen_dfa("conv_tri_trib2")

>>> Address automaton for block(subst(0->01, 1->02, 2->0), 2)
  DT polynomial: X^3-X^2-X-1 (n=3)
  θ=1.839286755214161
  DT automaton: 5 states, 5 final states, 9 transitions.

>>> address(block(subst(0->01, 1->02, 2->0), 2), 'trib2') is generating it's NS
  Numeration system automaton: 3 states, 3 final states, 5 transitions.
  Writing /var/folders/l8/fphf93t95171xj2_rf0knxtc0000gn/T/tmp.wBJ3QLoyq3/Walnut/Custom Bases/msd_trib2.txt in format Walnut
  +address(block(subst(0->01, 1->02, 2->0), 2), 'trib2')+address(block(subst(0->01, 1->02, 2->0), 2), 'trib2')-address(block(subst(0->01, 1->02, 2->0), 2), 'trib2') is generating it's dfa under name msd_trib2_addition
  Combination polynomial is X^3-X^2-X-1
  θ=1.839286755214161
  Combination automaton: 125 states, 125 final states, 729 transitions.
  (P,u) vector bound: (0.0, 0.0, 0.0) + (5.770539317125755, 8.360221985276333, 13.123391017013361)*C
  Fast poly bound: C in [(-1, -1, -1), (2, 2, 2)] => (34.62323590275453, 50.161331911657996, 

Translate the border condition predicate into the current numeration
system.

In [14]:
def bordercond2 "?msd_trib2 (?msd_tri Eii,jj,kk,nn 
($conv_tri_trib2(?msd_tri ii, ?msd_trib2 i) & 
$conv_tri_trib2(?msd_tri jj, ?msd_trib2 j) & 
$conv_tri_trib2(?msd_tri kk, ?msd_trib2 k) & 
$conv_tri_trib2(?msd_tri nn, ?msd_trib2 n) & 
$bordercond(ii,jj,kk,nn)))":






(conv_tri_trib2(ii,i))&conv_tri_trib2(jj,j))):9 states - 0ms
 ((conv_tri_trib2(ii,i))&conv_tri_trib2(jj,j)))&conv_tri_trib2(kk,k))):27 states - 0ms
  (((conv_tri_trib2(ii,i))&conv_tri_trib2(jj,j)))&conv_tri_trib2(kk,k)))&conv_tri_trib2(nn,n))):81 states - 1ms
   ((((conv_tri_trib2(ii,i))&conv_tri_trib2(jj,j)))&conv_tri_trib2(kk,k)))&conv_tri_trib2(nn,n)))&bordercond(ii,jj,kk,nn))):148 states - 2ms
    (E ii , jj , kk , nn ((((conv_tri_trib2(ii,i))&conv_tri_trib2(jj,j)))&conv_tri_trib2(kk,k)))&conv_tri_trib2(nn,n)))&bordercond(ii,jj,kk,nn)))):148 states - 1ms
Total computation time: 5ms.


### Compute Parikh vectors

First we compute the Parikh vectors for the prefixes of Trib2.

In [15]:
%%python
for (i,a) in enumerate(ns2.alpha):
    w = {'_': 0}
    w[a] = 1
    parikh = address(s2, ns2.ns, **w)
    (parikh - ns2).gen_dfa(f"trib2p{i}")

>>> Address automaton for block(subst(0->01, 1->02, 2->0), 2)
  DT polynomial: X^4-X^3-X^2-X (n=4)
  θ=1.839286755214161 (X^1)
  DT automaton: 5 states, 5 final states, 9 transitions.

>>> +address(block(subst(0->01, 1->02, 2->0), 2), 'trib2', '_': 0, 'a': 1)-address(block(subst(0->01, 1->02, 2->0), 2), 'trib2') is generating it's dfa under name trib2p0
  Combination polynomial is X^4-X^3-X^2-X
  θ=1.839286755214161 (X^1)
  Combination automaton: 25 states, 25 final states, 81 transitions.
  (P,u) vector bound: (0.0, 0.0, 0.0, 0.0) + (9.500520453722144, 13.578752712431406, 21.079791290562483, 34.876352197669426)*C
  Fast poly bound: C in [(-1, -2, -1, -1), (0, 0, 1, 1)] => (47.50260226861072, 67.89376356215703, 105.39895645281241, 174.38176098834714)
Expanding DFA...done.
  Flattened automaton : 1958 states, 21 final states, 3284 transitions.
  Trimmed flattened automaton : 34 states, 21 final states, 58 transitions.
  Real bound is [1, 2, 3, 6] (vs [47.50260226861072, 67.8937635621570

### Compute the complexity function

Use Walnut to generate a linear representation for the 2-abelian
complexity.

In [16]:
def fac0 "?msd_trib2 Ex,y $trib2p0(i,x) & $trib2p0(i+n,y) & z+x=y":
def fac1 "?msd_trib2 Ex,y $trib2p1(i,x) & $trib2p1(i+n,y) & z+x=y":
def fac2 "?msd_trib2 Ex,y $trib2p2(i,x) & $trib2p2(i+n,y) & z+x=y":
def fac3 "?msd_trib2 Ex,y $trib2p3(i,x) & $trib2p3(i+n,y) & z+x=y":
def fac4 "?msd_trib2 Ex,y $trib2p4(i,x) & $trib2p4(i+n,y) & z+x=y":

(trib2p0(i,x))&trib2p0((i+n),y))):678 states - 3ms
 (z+x)=y:149 states - 1ms
  ((trib2p0(i,x))&trib2p0((i+n),y)))&(z+x)=y):1427 states - 17ms
   (E x , y ((trib2p0(i,x))&trib2p0((i+n),y)))&(z+x)=y)):302 states - 3ms
Total computation time: 27ms.

(trib2p1(i,x))&trib2p1((i+n),y))):588 states - 1ms
 (z+x)=y:149 states - 1ms
  ((trib2p1(i,x))&trib2p1((i+n),y)))&(z+x)=y):1286 states - 12ms
   (E x , y ((trib2p1(i,x))&trib2p1((i+n),y)))&(z+x)=y)):283 states - 2ms
Total computation time: 19ms.

(trib2p2(i,x))&trib2p2((i+n),y))):1183 states - 4ms
 (z+x)=y:149 states - 1ms
  ((trib2p2(i,x))&trib2p2((i+n),y)))&(z+x)=y):3105 states - 39ms
   (E x , y ((trib2p2(i,x))&trib2p2((i+n),y)))&(z+x)=y)):431 states - 5ms
Total computation time: 53ms.

(trib2p3(i,x))&trib2p3((i+n),y))):973 states - 2ms
 (z+x)=y:149 states - 1ms
  ((trib2p3(i,x))&trib2p3((i+n),y)))&(z+x)=y):2672 states - 33ms
   (E x , y ((trib2p3(i,x))&trib2p3((i+n),y)))&(z+x)=y)):406 states - 4ms
Total computation time: 43ms.

(trib2p4(i,

In [17]:
def min0 "?msd_trib2 Ei $fac0(i,n,x) & Aj,y $fac0(j,n,y) => y>=x":
def min1 "?msd_trib2 Ei $fac1(i,n,x) & Aj,y $fac1(j,n,y) => y>=x":
def min2 "?msd_trib2 Ei $fac2(i,n,x) & Aj,y $fac2(j,n,y) => y>=x":
def min3 "?msd_trib2 Ei $fac3(i,n,x) & Aj,y $fac3(j,n,y) => y>=x":
def min4 "?msd_trib2 Ei $fac4(i,n,x) & Aj,y $fac4(j,n,y) => y>=x":

y>=x:12 states - 0ms
 (fac0(j,n,y))=>y>=x):1725 states - 52ms
  (A j , y (fac0(j,n,y))=>y>=x)):86 states - 57ms
   (fac0(i,n,x))&(A j , y (fac0(j,n,y))=>y>=x))):1190 states - 3ms
    (E i (fac0(i,n,x))&(A j , y (fac0(j,n,y))=>y>=x)))):98 states - 2ms
Total computation time: 116ms.

y>=x:12 states - 0ms
 (fac1(j,n,y))=>y>=x):1690 states - 52ms
  (A j , y (fac1(j,n,y))=>y>=x)):86 states - 56ms
   (fac1(i,n,x))&(A j , y (fac1(j,n,y))=>y>=x))):1174 states - 2ms
    (E i (fac1(i,n,x))&(A j , y (fac1(j,n,y))=>y>=x)))):98 states - 2ms
Total computation time: 114ms.

y>=x:12 states - 0ms
 (fac2(j,n,y))=>y>=x):2568 states - 79ms
  (A j , y (fac2(j,n,y))=>y>=x)):97 states - 87ms
   (fac2(i,n,x))&(A j , y (fac2(j,n,y))=>y>=x))):1349 states - 3ms
    (E i (fac2(i,n,x))&(A j , y (fac2(j,n,y))=>y>=x)))):113 states - 3ms
Total computation time: 174ms.

y>=x:12 states - 0ms
 (fac3(j,n,y))=>y>=x):2515 states - 75ms
  (A j , y (fac3(j,n,y))=>y>=x)):97 states - 78ms
   (fac3(i,n,x))&(A j , y (fac3(j,n,y)

In [18]:
def diff0 "?msd_trib2 Ex,y $min0(n,x) & $fac0(i,n,y) & z+x=y":
def diff1 "?msd_trib2 Ex,y $min1(n,x) & $fac1(i,n,y) & z+x=y":
def diff2 "?msd_trib2 Ex,y $min2(n,x) & $fac2(i,n,y) & z+x=y":
def diff3 "?msd_trib2 Ex,y $min3(n,x) & $fac3(i,n,y) & z+x=y":
def diff4 "?msd_trib2 Ex,y $min4(n,x) & $fac4(i,n,y) & z+x=y":

(min0(n,x))&fac0(i,n,y))):1703 states - 6ms
 (z+x)=y:149 states - 1ms
  ((min0(n,x))&fac0(i,n,y)))&(z+x)=y):1672 states - 7ms
   (E x , y ((min0(n,x))&fac0(i,n,y)))&(z+x)=y)):1090 states - 6ms
Total computation time: 22ms.

(min1(n,x))&fac1(i,n,y))):1615 states - 6ms
 (z+x)=y:149 states - 1ms
  ((min1(n,x))&fac1(i,n,y)))&(z+x)=y):1611 states - 7ms
   (E x , y ((min1(n,x))&fac1(i,n,y)))&(z+x)=y)):1088 states - 6ms
Total computation time: 21ms.

(min2(n,x))&fac2(i,n,y))):2019 states - 8ms
 (z+x)=y:149 states - 1ms
  ((min2(n,x))&fac2(i,n,y)))&(z+x)=y):1999 states - 9ms
   (E x , y ((min2(n,x))&fac2(i,n,y)))&(z+x)=y)):1093 states - 7ms
Total computation time: 26ms.

(min3(n,x))&fac3(i,n,y))):1931 states - 6ms
 (z+x)=y:149 states - 1ms
  ((min3(n,x))&fac3(i,n,y)))&(z+x)=y):1920 states - 8ms
   (E x , y ((min3(n,x))&fac3(i,n,y)))&(z+x)=y)):1089 states - 6ms
Total computation time: 23ms.

(min4(n,x))&fac4(i,n,y))):2500 states - 9ms
 (z+x)=y:149 states - 1ms
  ((min4(n,x))&fac4(i,n,y)))&(z+x)

In [19]:
def abeq_trib2 "?msd_trib2 $bordercond2(i,j,2,n+1) & (Ez $diff0(i,n,z) &  $diff0(j,n,z)) & (Ez $diff1(i,n,z) &  $diff1(j,n,z)) & (Ez $diff2(i,n,z) &  $diff2(j,n,z)) & (Ez $diff3(i,n,z) &  $diff3(j,n,z)) & (Ez $diff4(i,n,z) &  $diff4(j,n,z))":
def abeq_tri2 "?msd_tri (n<1 & $feq_tri(i,j,n)) | (n>=1 & (?msd_trib2 Eii,jj,nn ($conv_tri_trib2(?msd_tri i, ?msd_trib2 ii) & $conv_tri_trib2(?msd_tri j, ?msd_trib2 jj) & $conv_tri_trib2(?msd_tri n, ?msd_trib2 nn) & $abeq_trib2(ii,jj,nn-1))))":

computed ~:1 states - 1ms
computed ~:2 states - 0ms
(diff0(i,n,z))&diff0(j,n,z))):17020 states - 91ms
 (E z (diff0(i,n,z))&diff0(j,n,z)))):2613 states - 52ms
  (bordercond2(i,j,2,(n+1)))&(E z (diff0(i,n,z))&diff0(j,n,z))))):2613 states - 8ms
   (diff1(i,n,z))&diff1(j,n,z))):17010 states - 91ms
    (E z (diff1(i,n,z))&diff1(j,n,z)))):2613 states - 53ms
     ((bordercond2(i,j,2,(n+1)))&(E z (diff0(i,n,z))&diff0(j,n,z)))))&(E z (diff1(i,n,z))&diff1(j,n,z))))):2613 states - 8ms
      (diff2(i,n,z))&diff2(j,n,z))):17033 states - 94ms
       (E z (diff2(i,n,z))&diff2(j,n,z)))):2613 states - 53ms
        (((bordercond2(i,j,2,(n+1)))&(E z (diff0(i,n,z))&diff0(j,n,z)))))&(E z (diff1(i,n,z))&diff1(j,n,z)))))&(E z (diff2(i,n,z))&diff2(j,n,z))))):2613 states - 8ms
         (diff3(i,n,z))&diff3(j,n,z))):17015 states - 91ms
          (E z (diff3(i,n,z))&diff3(j,n,z)))):2613 states - 58ms
           ((((bordercond2(i,j,2,(n+1)))&(E z (diff0(i,n,z))&diff0(j,n,z)))))&(E z (diff1(i,n,z))&diff1(j,n,z))))

In [20]:
eval comp_tri2 n "?msd_tri Aj $abeq_tri2(i,j,n) => i<=j":

i<=j:12 states - 1ms
 (abeq_tri2(i,j,n))=>i<=j):3503 states - 61ms
  (A j (abeq_tri2(i,j,n))=>i<=j)):251 states - 80ms
Total computation time: 154ms.


### Apply the semigroup trick

In [21]:
%SGT comp_tri2 msd_tri Comp_tri2

Reading Result/comp_tri2.mpl
Rank: 72
168 states
values: [0, 1, 3, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21]
Writing Word Automata Library/Comp_tri2.txt
Done.

## 3-abelian complexity

Define the 3-block map of tri and construct the conversion predicate
between both numeration systems.

In [22]:
%%python
s3 = block(s, 3)
ns3 = address(s3, "trib3")
ns3.gen_ns()
(ns-ns3).gen_dfa("conv_tri_trib3")

>>> Address automaton for block(subst(0->01, 1->02, 2->0), 3)
  DT polynomial: X^3-X^2-X-1 (n=3)
  θ=1.839286755214161
  DT automaton: 7 states, 7 final states, 13 transitions.

>>> address(block(subst(0->01, 1->02, 2->0), 3), 'trib3') is generating it's NS
  Numeration system automaton: 3 states, 3 final states, 5 transitions.
  Writing /var/folders/l8/fphf93t95171xj2_rf0knxtc0000gn/T/tmp.wBJ3QLoyq3/Walnut/Custom Bases/msd_trib3.txt in format Walnut
  +address(block(subst(0->01, 1->02, 2->0), 3), 'trib3')+address(block(subst(0->01, 1->02, 2->0), 3), 'trib3')-address(block(subst(0->01, 1->02, 2->0), 3), 'trib3') is generating it's dfa under name msd_trib3_addition
  Combination polynomial is X^3-X^2-X-1
  θ=1.839286755214161
  Combination automaton: 343 states, 343 final states, 2197 transitions.
  (P,u) vector bound: (0.0, 0.0, 0.0) + (5.770539317125755, 8.360221985276333, 13.123391017013361)*C
  Fast poly bound: C in [(-1, -1, -1), (2, 2, 2)] => (34.62323590275453, 50.161331911657996

Translate the border condition predicate into the current numeration
system.

In [23]:
def bordercond3 "?msd_trib3 (?msd_tri Eii,jj,kk,nn 
($conv_tri_trib3(?msd_tri ii, ?msd_trib3 i) & 
$conv_tri_trib3(?msd_tri jj, ?msd_trib3 j) & 
$conv_tri_trib3(?msd_tri kk, ?msd_trib3 k) & 
$conv_tri_trib3(?msd_tri nn, ?msd_trib3 n) & 
$bordercond(ii,jj,kk,nn)))":






(conv_tri_trib3(ii,i))&conv_tri_trib3(jj,j))):9 states - 0ms
 ((conv_tri_trib3(ii,i))&conv_tri_trib3(jj,j)))&conv_tri_trib3(kk,k))):27 states - 0ms
  (((conv_tri_trib3(ii,i))&conv_tri_trib3(jj,j)))&conv_tri_trib3(kk,k)))&conv_tri_trib3(nn,n))):81 states - 1ms
   ((((conv_tri_trib3(ii,i))&conv_tri_trib3(jj,j)))&conv_tri_trib3(kk,k)))&conv_tri_trib3(nn,n)))&bordercond(ii,jj,kk,nn))):148 states - 3ms
    (E ii , jj , kk , nn ((((conv_tri_trib3(ii,i))&conv_tri_trib3(jj,j)))&conv_tri_trib3(kk,k)))&conv_tri_trib3(nn,n)))&bordercond(ii,jj,kk,nn)))):148 states - 1ms
Total computation time: 5ms.


### Compute Parikh vectors

First we compute the Parikh vectors for the prefixes of Trib3.

In [24]:
%%python
for (i,a) in enumerate(ns3.alpha):
    w = {'_': 0}
    w[a] = 1
    parikh = address(s3, ns3.ns, **w)
    (parikh - ns3).gen_dfa(f"trib3p{i}")

>>> Address automaton for block(subst(0->01, 1->02, 2->0), 3)
  DT polynomial: X^4-X^3-X^2-X (n=4)
  θ=1.839286755214161 (X^1)
  DT automaton: 7 states, 7 final states, 13 transitions.

>>> +address(block(subst(0->01, 1->02, 2->0), 3), 'trib3', '_': 0, 'a': 1)-address(block(subst(0->01, 1->02, 2->0), 3), 'trib3') is generating it's dfa under name trib3p0
  Combination polynomial is X^4-X^3-X^2-X
  θ=1.839286755214161 (X^1)
  Combination automaton: 49 states, 49 final states, 169 transitions.
  (P,u) vector bound: (0.0, 0.0, 0.0, 0.0) + (9.500520453722144, 13.578752712431406, 21.079791290562483, 34.876352197669426)*C
  Fast poly bound: C in [(-1, -2, -1, -1), (0, 0, 1, 1)] => (47.50260226861072, 67.89376356215703, 105.39895645281241, 174.38176098834714)
Expanding DFA...done.
  Flattened automaton : 2613 states, 29 final states, 4406 transitions.
  Trimmed flattened automaton : 46 states, 29 final states, 80 transitions.
  Real bound is [1, 2, 3, 6] (vs [47.50260226861072, 67.89376356215

### Compute the complexity function

Use Walnut to generate a linear representation for the 3-abelian
complexity.

In [25]:
def fac0 "?msd_trib3 Ex,y $trib3p0(i,x) & $trib3p0(i+n,y) & z+x=y":
def fac1 "?msd_trib3 Ex,y $trib3p1(i,x) & $trib3p1(i+n,y) & z+x=y":
def fac2 "?msd_trib3 Ex,y $trib3p2(i,x) & $trib3p2(i+n,y) & z+x=y":
def fac3 "?msd_trib3 Ex,y $trib3p3(i,x) & $trib3p3(i+n,y) & z+x=y":
def fac4 "?msd_trib3 Ex,y $trib3p4(i,x) & $trib3p4(i+n,y) & z+x=y":
def fac5 "?msd_trib3 Ex,y $trib3p5(i,x) & $trib3p5(i+n,y) & z+x=y":
def fac6 "?msd_trib3 Ex,y $trib3p6(i,x) & $trib3p6(i+n,y) & z+x=y":

(trib3p0(i,x))&trib3p0((i+n),y))):678 states - 2ms
 (z+x)=y:149 states - 1ms
  ((trib3p0(i,x))&trib3p0((i+n),y)))&(z+x)=y):1427 states - 16ms
   (E x , y ((trib3p0(i,x))&trib3p0((i+n),y)))&(z+x)=y)):302 states - 2ms
Total computation time: 24ms.

(trib3p1(i,x))&trib3p1((i+n),y))):1265 states - 5ms
 (z+x)=y:149 states - 1ms
  ((trib3p1(i,x))&trib3p1((i+n),y)))&(z+x)=y):3294 states - 46ms
   (E x , y ((trib3p1(i,x))&trib3p1((i+n),y)))&(z+x)=y)):611 states - 6ms
Total computation time: 63ms.

(trib3p2(i,x))&trib3p2((i+n),y))):1183 states - 4ms
 (z+x)=y:149 states - 1ms
  ((trib3p2(i,x))&trib3p2((i+n),y)))&(z+x)=y):3105 states - 42ms
   (E x , y ((trib3p2(i,x))&trib3p2((i+n),y)))&(z+x)=y)):431 states - 5ms
Total computation time: 56ms.

(trib3p3(i,x))&trib3p3((i+n),y))):973 states - 3ms
 (z+x)=y:149 states - 1ms
  ((trib3p3(i,x))&trib3p3((i+n),y)))&(z+x)=y):2672 states - 34ms
   (E x , y ((trib3p3(i,x))&trib3p3((i+n),y)))&(z+x)=y)):406 states - 4ms
Total computation time: 45ms.

(trib3p4(i

In [26]:
def min0 "?msd_trib3 Ei $fac0(i,n,x) & Aj,y $fac0(j,n,y) => y>=x":
def min1 "?msd_trib3 Ei $fac1(i,n,x) & Aj,y $fac1(j,n,y) => y>=x":
def min2 "?msd_trib3 Ei $fac2(i,n,x) & Aj,y $fac2(j,n,y) => y>=x":
def min3 "?msd_trib3 Ei $fac3(i,n,x) & Aj,y $fac3(j,n,y) => y>=x":
def min4 "?msd_trib3 Ei $fac4(i,n,x) & Aj,y $fac4(j,n,y) => y>=x":
def min5 "?msd_trib3 Ei $fac5(i,n,x) & Aj,y $fac5(j,n,y) => y>=x":
def min6 "?msd_trib3 Ei $fac6(i,n,x) & Aj,y $fac6(j,n,y) => y>=x":

y>=x:12 states - 0ms
 (fac0(j,n,y))=>y>=x):1725 states - 50ms
  (A j , y (fac0(j,n,y))=>y>=x)):86 states - 52ms
   (fac0(i,n,x))&(A j , y (fac0(j,n,y))=>y>=x))):1190 states - 3ms
    (E i (fac0(i,n,x))&(A j , y (fac0(j,n,y))=>y>=x)))):98 states - 2ms
Total computation time: 109ms.

y>=x:12 states - 0ms
 (fac1(j,n,y))=>y>=x):3341 states - 88ms
  (A j , y (fac1(j,n,y))=>y>=x)):97 states - 107ms
   (fac1(i,n,x))&(A j , y (fac1(j,n,y))=>y>=x))):1773 states - 4ms
    (E i (fac1(i,n,x))&(A j , y (fac1(j,n,y))=>y>=x)))):113 states - 3ms
Total computation time: 205ms.

y>=x:12 states - 0ms
 (fac2(j,n,y))=>y>=x):2568 states - 70ms
  (A j , y (fac2(j,n,y))=>y>=x)):97 states - 75ms
   (fac2(i,n,x))&(A j , y (fac2(j,n,y))=>y>=x))):1349 states - 3ms
    (E i (fac2(i,n,x))&(A j , y (fac2(j,n,y))=>y>=x)))):113 states - 2ms
Total computation time: 153ms.

y>=x:12 states - 0ms
 (fac3(j,n,y))=>y>=x):2515 states - 68ms
  (A j , y (fac3(j,n,y))=>y>=x)):97 states - 74ms
   (fac3(i,n,x))&(A j , y (fac3(j,n,

In [27]:
def diff0 "?msd_trib3 Ex,y $min0(n,x) & $fac0(i,n,y) & z+x=y":
def diff1 "?msd_trib3 Ex,y $min1(n,x) & $fac1(i,n,y) & z+x=y":
def diff2 "?msd_trib3 Ex,y $min2(n,x) & $fac2(i,n,y) & z+x=y":
def diff3 "?msd_trib3 Ex,y $min3(n,x) & $fac3(i,n,y) & z+x=y":
def diff4 "?msd_trib3 Ex,y $min4(n,x) & $fac4(i,n,y) & z+x=y":
def diff5 "?msd_trib3 Ex,y $min5(n,x) & $fac5(i,n,y) & z+x=y":
def diff6 "?msd_trib3 Ex,y $min6(n,x) & $fac6(i,n,y) & z+x=y":

(min0(n,x))&fac0(i,n,y))):1703 states - 7ms
 (z+x)=y:149 states - 1ms
  ((min0(n,x))&fac0(i,n,y)))&(z+x)=y):1672 states - 7ms
   (E x , y ((min0(n,x))&fac0(i,n,y)))&(z+x)=y)):1090 states - 6ms
Total computation time: 23ms.

(min1(n,x))&fac1(i,n,y))):2678 states - 12ms
 (z+x)=y:149 states - 1ms
  ((min1(n,x))&fac1(i,n,y)))&(z+x)=y):2555 states - 12ms
   (E x , y ((min1(n,x))&fac1(i,n,y)))&(z+x)=y)):1538 states - 8ms
Total computation time: 36ms.

(min2(n,x))&fac2(i,n,y))):2019 states - 8ms
 (z+x)=y:149 states - 1ms
  ((min2(n,x))&fac2(i,n,y)))&(z+x)=y):1999 states - 9ms
   (E x , y ((min2(n,x))&fac2(i,n,y)))&(z+x)=y)):1093 states - 6ms
Total computation time: 27ms.

(min3(n,x))&fac3(i,n,y))):1931 states - 8ms
 (z+x)=y:149 states - 1ms
  ((min3(n,x))&fac3(i,n,y)))&(z+x)=y):1920 states - 8ms
   (E x , y ((min3(n,x))&fac3(i,n,y)))&(z+x)=y)):1089 states - 6ms
Total computation time: 25ms.

(min4(n,x))&fac4(i,n,y))):2500 states - 9ms
 (z+x)=y:149 states - 1ms
  ((min4(n,x))&fac4(i,n,y)))&(z+

In [28]:
def abeq_trib3 "?msd_trib3 $bordercond3(i,j,3,n+2) & (Ez $diff0(i,n,z) &  $diff0(j,n,z)) & (Ez $diff1(i,n,z) &  $diff1(j,n,z)) & (Ez $diff2(i,n,z) &  $diff2(j,n,z)) & (Ez $diff3(i,n,z) &  $diff3(j,n,z)) & (Ez $diff4(i,n,z) &  $diff4(j,n,z)) & (Ez $diff5(i,n,z) &  $diff5(j,n,z)) & (Ez $diff6(i,n,z) &  $diff6(j,n,z))":
def abeq_tri3 "?msd_tri (n<2 & $feq_tri(i,j,n)) | (n>=2 & (?msd_trib3 Eii,jj,nn ($conv_tri_trib3(?msd_tri i, ?msd_trib3 ii) & $conv_tri_trib3(?msd_tri j, ?msd_trib3 jj) & $conv_tri_trib3(?msd_tri n, ?msd_trib3 nn) & $abeq_trib3(ii,jj,nn-2))))":

computed ~:1 states - 1ms
computed ~:2 states - 0ms
(diff0(i,n,z))&diff0(j,n,z))):17020 states - 87ms
 (E z (diff0(i,n,z))&diff0(j,n,z)))):2613 states - 49ms
  (bordercond3(i,j,3,(n+2)))&(E z (diff0(i,n,z))&diff0(j,n,z))))):2795 states - 9ms
   (diff1(i,n,z))&diff1(j,n,z))):26522 states - 186ms
    (E z (diff1(i,n,z))&diff1(j,n,z)))):5140 states - 106ms
     ((bordercond3(i,j,3,(n+2)))&(E z (diff0(i,n,z))&diff0(j,n,z)))))&(E z (diff1(i,n,z))&diff1(j,n,z))))):3080 states - 15ms
      (diff2(i,n,z))&diff2(j,n,z))):17033 states - 82ms
       (E z (diff2(i,n,z))&diff2(j,n,z)))):2613 states - 48ms
        (((bordercond3(i,j,3,(n+2)))&(E z (diff0(i,n,z))&diff0(j,n,z)))))&(E z (diff1(i,n,z))&diff1(j,n,z)))))&(E z (diff2(i,n,z))&diff2(j,n,z))))):2928 states - 9ms
         (diff3(i,n,z))&diff3(j,n,z))):17015 states - 91ms
          (E z (diff3(i,n,z))&diff3(j,n,z)))):2613 states - 54ms
           ((((bordercond3(i,j,3,(n+2)))&(E z (diff0(i,n,z))&diff0(j,n,z)))))&(E z (diff1(i,n,z))&diff1(j,n,z)

In [29]:
eval comp_tri3 n "?msd_tri Aj $abeq_tri3(i,j,n) => i<=j":

i<=j:12 states - 0ms
 (abeq_tri3(i,j,n))=>i<=j):3845 states - 53ms
  (A j (abeq_tri3(i,j,n))=>i<=j)):308 states - 84ms
Total computation time: 149ms.


### Apply the semigroup trick

In [30]:
%SGT comp_tri3 msd_tri Comp_tri3

Reading Result/comp_tri3.mpl
Rank: 78
263 states
values: [0, 1, 3, 5, 7, 9, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 35]
Writing Word Automata Library/Comp_tri3.txt
Done.

## 4-abelian complexity

Define the 4-block map of tri and construct the conversion predicate
between both numeration systems.

In [31]:
%%python
s4 = block(s, 4)
ns4 = address(s4, "trib4")
ns4.gen_ns()
(ns-ns4).gen_dfa("conv_tri_trib4")

>>> Address automaton for block(subst(0->01, 1->02, 2->0), 4)
  DT polynomial: X^3-X^2-X-1 (n=3)
  θ=1.839286755214161
  DT automaton: 9 states, 9 final states, 17 transitions.

>>> address(block(subst(0->01, 1->02, 2->0), 4), 'trib4') is generating it's NS
  Numeration system automaton: 3 states, 3 final states, 5 transitions.
  Writing /var/folders/l8/fphf93t95171xj2_rf0knxtc0000gn/T/tmp.wBJ3QLoyq3/Walnut/Custom Bases/msd_trib4.txt in format Walnut
  +address(block(subst(0->01, 1->02, 2->0), 4), 'trib4')+address(block(subst(0->01, 1->02, 2->0), 4), 'trib4')-address(block(subst(0->01, 1->02, 2->0), 4), 'trib4') is generating it's dfa under name msd_trib4_addition
  Combination polynomial is X^3-X^2-X-1
  θ=1.839286755214161
  Combination automaton: 729 states, 729 final states, 4913 transitions.
  (P,u) vector bound: (0.0, 0.0, 0.0) + (5.770539317125755, 8.360221985276333, 13.123391017013361)*C
  Fast poly bound: C in [(-1, -1, -1), (2, 2, 2)] => (34.62323590275453, 50.161331911657996

Translate the border condition predicate into the current numeration
system.

In [32]:
def bordercond4 "?msd_trib4 (?msd_tri Eii,jj,kk,nn 
($conv_tri_trib4(?msd_tri ii, ?msd_trib4 i) & 
$conv_tri_trib4(?msd_tri jj, ?msd_trib4 j) & 
$conv_tri_trib4(?msd_tri kk, ?msd_trib4 k) & 
$conv_tri_trib4(?msd_tri nn, ?msd_trib4 n) & 
$bordercond(ii,jj,kk,nn)))":






(conv_tri_trib4(ii,i))&conv_tri_trib4(jj,j))):9 states - 0ms
 ((conv_tri_trib4(ii,i))&conv_tri_trib4(jj,j)))&conv_tri_trib4(kk,k))):27 states - 0ms
  (((conv_tri_trib4(ii,i))&conv_tri_trib4(jj,j)))&conv_tri_trib4(kk,k)))&conv_tri_trib4(nn,n))):81 states - 1ms
   ((((conv_tri_trib4(ii,i))&conv_tri_trib4(jj,j)))&conv_tri_trib4(kk,k)))&conv_tri_trib4(nn,n)))&bordercond(ii,jj,kk,nn))):148 states - 2ms
    (E ii , jj , kk , nn ((((conv_tri_trib4(ii,i))&conv_tri_trib4(jj,j)))&conv_tri_trib4(kk,k)))&conv_tri_trib4(nn,n)))&bordercond(ii,jj,kk,nn)))):148 states - 1ms
Total computation time: 5ms.


### Compute Parikh vectors

First we compute the Parikh vectors for the prefixes of Trib4.

In [33]:
%%python
for (i,a) in enumerate(ns4.alpha):
    w = {'_': 0}
    w[a] = 1
    parikh = address(s4, ns4.ns, **w)
    (parikh - ns4).gen_dfa(f"trib4p{i}")

>>> Address automaton for block(subst(0->01, 1->02, 2->0), 4)
  DT polynomial: X^5-X^4-X^3-X^2 (n=5)
  θ=1.839286755214161 (X^2)
  DT automaton: 9 states, 9 final states, 17 transitions.

>>> +address(block(subst(0->01, 1->02, 2->0), 4), 'trib4', '_': 0, 'a': 1)-address(block(subst(0->01, 1->02, 2->0), 4), 'trib4') is generating it's dfa under name trib4p0
  Combination polynomial is X^5-X^4-X^3-X^2
  θ=1.839286755214161 (X^2)
  Combination automaton: 81 states, 81 final states, 289 transitions.
  (P,u) vector bound: (0.0, 0.0, 0.0, 0.0, 0.0) + (13.995256614112588, 19.917967506314703, 30.811531205304703, 50.84791863393879, 87.70058065376499)*C
  Fast poly bound: C in [(-1, -2, -2, -1, -1), (0, 0, 1, 1, 1)] => (97.96679629878811, 139.4257725442029, 215.68071843713292, 355.93543043757154, 613.9040645763549)
Expanding DFA...done.
  Flattened automaton : 10627 states, 63 final states, 18842 transitions.
  Trimmed flattened automaton : 114 states, 63 final states, 208 transitions.
  Real bo

### Compute the complexity function

Use Walnut to generate a linear representation for the 4-abelian
complexity.

In [34]:
def fac0 "?msd_trib4 Ex,y $trib4p0(i,x) & $trib4p0(i+n,y) & z+x=y":
def fac1 "?msd_trib4 Ex,y $trib4p1(i,x) & $trib4p1(i+n,y) & z+x=y":
def fac2 "?msd_trib4 Ex,y $trib4p2(i,x) & $trib4p2(i+n,y) & z+x=y":
def fac3 "?msd_trib4 Ex,y $trib4p3(i,x) & $trib4p3(i+n,y) & z+x=y":
def fac4 "?msd_trib4 Ex,y $trib4p4(i,x) & $trib4p4(i+n,y) & z+x=y":
def fac5 "?msd_trib4 Ex,y $trib4p5(i,x) & $trib4p5(i+n,y) & z+x=y":
def fac6 "?msd_trib4 Ex,y $trib4p6(i,x) & $trib4p6(i+n,y) & z+x=y":
def fac7 "?msd_trib4 Ex,y $trib4p7(i,x) & $trib4p7(i+n,y) & z+x=y":
def fac8 "?msd_trib4 Ex,y $trib4p8(i,x) & $trib4p8(i+n,y) & z+x=y":

(trib4p0(i,x))&trib4p0((i+n),y))):1385 states - 5ms
 (z+x)=y:149 states - 1ms
  ((trib4p0(i,x))&trib4p0((i+n),y)))&(z+x)=y):3569 states - 50ms
   (E x , y ((trib4p0(i,x))&trib4p0((i+n),y)))&(z+x)=y)):873 states - 7ms
Total computation time: 67ms.

(trib4p1(i,x))&trib4p1((i+n),y))):1265 states - 5ms
 (z+x)=y:149 states - 1ms
  ((trib4p1(i,x))&trib4p1((i+n),y)))&(z+x)=y):3294 states - 40ms
   (E x , y ((trib4p1(i,x))&trib4p1((i+n),y)))&(z+x)=y)):611 states - 6ms
Total computation time: 57ms.

(trib4p2(i,x))&trib4p2((i+n),y))):1183 states - 4ms
 (z+x)=y:149 states - 1ms
  ((trib4p2(i,x))&trib4p2((i+n),y)))&(z+x)=y):3105 states - 37ms
   (E x , y ((trib4p2(i,x))&trib4p2((i+n),y)))&(z+x)=y)):431 states - 5ms
Total computation time: 52ms.

(trib4p3(i,x))&trib4p3((i+n),y))):973 states - 2ms
 (z+x)=y:149 states - 1ms
  ((trib4p3(i,x))&trib4p3((i+n),y)))&(z+x)=y):2672 states - 31ms
   (E x , y ((trib4p3(i,x))&trib4p3((i+n),y)))&(z+x)=y)):406 states - 4ms
Total computation time: 43ms.

(trib4p4(

In [35]:
def min0 "?msd_trib4 Ei $fac0(i,n,x) & Aj,y $fac0(j,n,y) => y>=x":
def min1 "?msd_trib4 Ei $fac1(i,n,x) & Aj,y $fac1(j,n,y) => y>=x":
def min2 "?msd_trib4 Ei $fac2(i,n,x) & Aj,y $fac2(j,n,y) => y>=x":
def min3 "?msd_trib4 Ei $fac3(i,n,x) & Aj,y $fac3(j,n,y) => y>=x":
def min4 "?msd_trib4 Ei $fac4(i,n,x) & Aj,y $fac4(j,n,y) => y>=x":
def min5 "?msd_trib4 Ei $fac5(i,n,x) & Aj,y $fac5(j,n,y) => y>=x":
def min6 "?msd_trib4 Ei $fac6(i,n,x) & Aj,y $fac6(j,n,y) => y>=x":
def min7 "?msd_trib4 Ei $fac7(i,n,x) & Aj,y $fac7(j,n,y) => y>=x":
def min8 "?msd_trib4 Ei $fac8(i,n,x) & Aj,y $fac8(j,n,y) => y>=x":

y>=x:12 states - 0ms
 (fac0(j,n,y))=>y>=x):4486 states - 117ms
  (A j , y (fac0(j,n,y))=>y>=x)):97 states - 146ms
   (fac0(i,n,x))&(A j , y (fac0(j,n,y))=>y>=x))):2203 states - 6ms
    (E i (fac0(i,n,x))&(A j , y (fac0(j,n,y))=>y>=x)))):113 states - 4ms
Total computation time: 277ms.

y>=x:12 states - 0ms
 (fac1(j,n,y))=>y>=x):3341 states - 102ms
  (A j , y (fac1(j,n,y))=>y>=x)):97 states - 125ms
   (fac1(i,n,x))&(A j , y (fac1(j,n,y))=>y>=x))):1773 states - 6ms
    (E i (fac1(i,n,x))&(A j , y (fac1(j,n,y))=>y>=x)))):113 states - 4ms
Total computation time: 240ms.

y>=x:12 states - 0ms
 (fac2(j,n,y))=>y>=x):2568 states - 74ms
  (A j , y (fac2(j,n,y))=>y>=x)):97 states - 75ms
   (fac2(i,n,x))&(A j , y (fac2(j,n,y))=>y>=x))):1349 states - 3ms
    (E i (fac2(i,n,x))&(A j , y (fac2(j,n,y))=>y>=x)))):113 states - 2ms
Total computation time: 157ms.

y>=x:12 states - 0ms
 (fac3(j,n,y))=>y>=x):2515 states - 68ms
  (A j , y (fac3(j,n,y))=>y>=x)):97 states - 74ms
   (fac3(i,n,x))&(A j , y (fac3(

In [36]:
def diff0 "?msd_trib4 Ex,y $min0(n,x) & $fac0(i,n,y) & z+x=y":
def diff1 "?msd_trib4 Ex,y $min1(n,x) & $fac1(i,n,y) & z+x=y":
def diff2 "?msd_trib4 Ex,y $min2(n,x) & $fac2(i,n,y) & z+x=y":
def diff3 "?msd_trib4 Ex,y $min3(n,x) & $fac3(i,n,y) & z+x=y":
def diff4 "?msd_trib4 Ex,y $min4(n,x) & $fac4(i,n,y) & z+x=y":
def diff5 "?msd_trib4 Ex,y $min5(n,x) & $fac5(i,n,y) & z+x=y":
def diff6 "?msd_trib4 Ex,y $min6(n,x) & $fac6(i,n,y) & z+x=y":
def diff7 "?msd_trib4 Ex,y $min7(n,x) & $fac7(i,n,y) & z+x=y":
def diff8 "?msd_trib4 Ex,y $min8(n,x) & $fac8(i,n,y) & z+x=y":

(min0(n,x))&fac0(i,n,y))):3534 states - 15ms
 (z+x)=y:149 states - 1ms
  ((min0(n,x))&fac0(i,n,y)))&(z+x)=y):3346 states - 16ms
   (E x , y ((min0(n,x))&fac0(i,n,y)))&(z+x)=y)):2046 states - 13ms
Total computation time: 48ms.

(min1(n,x))&fac1(i,n,y))):2678 states - 11ms
 (z+x)=y:149 states - 1ms
  ((min1(n,x))&fac1(i,n,y)))&(z+x)=y):2555 states - 11ms
   (E x , y ((min1(n,x))&fac1(i,n,y)))&(z+x)=y)):1538 states - 9ms
Total computation time: 34ms.

(min2(n,x))&fac2(i,n,y))):2019 states - 8ms
 (z+x)=y:149 states - 1ms
  ((min2(n,x))&fac2(i,n,y)))&(z+x)=y):1999 states - 9ms
   (E x , y ((min2(n,x))&fac2(i,n,y)))&(z+x)=y)):1093 states - 7ms
Total computation time: 26ms.

(min3(n,x))&fac3(i,n,y))):1931 states - 7ms
 (z+x)=y:149 states - 1ms
  ((min3(n,x))&fac3(i,n,y)))&(z+x)=y):1920 states - 8ms
   (E x , y ((min3(n,x))&fac3(i,n,y)))&(z+x)=y)):1089 states - 7ms
Total computation time: 24ms.

(min4(n,x))&fac4(i,n,y))):2500 states - 9ms
 (z+x)=y:149 states - 1ms
  ((min4(n,x))&fac4(i,n,y)))&

In [37]:
def abeq_trib4 "?msd_trib4 $bordercond4(i,j,4,n+3) & (Ez $diff0(i,n,z) &  $diff0(j,n,z)) & (Ez $diff1(i,n,z) &  $diff1(j,n,z)) & (Ez $diff2(i,n,z) &  $diff2(j,n,z)) & (Ez $diff3(i,n,z) &  $diff3(j,n,z)) & (Ez $diff4(i,n,z) &  $diff4(j,n,z)) & (Ez $diff5(i,n,z) &  $diff5(j,n,z)) & (Ez $diff6(i,n,z) &  $diff6(j,n,z)) & (Ez $diff7(i,n,z) &  $diff7(j,n,z)) & (Ez $diff8(i,n,z) &  $diff8(j,n,z))":
def abeq_tri4 "?msd_tri (n<3 & $feq_tri(i,j,n)) | (n>=3 & (?msd_trib4 Eii,jj,nn ($conv_tri_trib4(?msd_tri i, ?msd_trib4 ii) & $conv_tri_trib4(?msd_tri j, ?msd_trib4 jj) & $conv_tri_trib4(?msd_tri n, ?msd_trib4 nn) & $abeq_trib4(ii,jj,nn-3))))":

computed ~:1 states - 1ms
computed ~:2 states - 0ms
(diff0(i,n,z))&diff0(j,n,z))):44965 states - 465ms
 (E z (diff0(i,n,z))&diff0(j,n,z)))):10392 states - 399ms
  (bordercond4(i,j,4,(n+3)))&(E z (diff0(i,n,z))&diff0(j,n,z))))):4189 states - 36ms
   (diff1(i,n,z))&diff1(j,n,z))):26522 states - 214ms
    (E z (diff1(i,n,z))&diff1(j,n,z)))):5140 states - 128ms
     ((bordercond4(i,j,4,(n+3)))&(E z (diff0(i,n,z))&diff0(j,n,z)))))&(E z (diff1(i,n,z))&diff1(j,n,z))))):3845 states - 21ms
      (diff2(i,n,z))&diff2(j,n,z))):17033 states - 92ms
       (E z (diff2(i,n,z))&diff2(j,n,z)))):2613 states - 53ms
        (((bordercond4(i,j,4,(n+3)))&(E z (diff0(i,n,z))&diff0(j,n,z)))))&(E z (diff1(i,n,z))&diff1(j,n,z)))))&(E z (diff2(i,n,z))&diff2(j,n,z))))):3557 states - 13ms
         (diff3(i,n,z))&diff3(j,n,z))):17015 states - 83ms
          (E z (diff3(i,n,z))&diff3(j,n,z)))):2613 states - 49ms
           ((((bordercond4(i,j,4,(n+3)))&(E z (diff0(i,n,z))&diff0(j,n,z)))))&(E z (diff1(i,n,z))&diff1(j

In [38]:
eval comp_tri4 n "?msd_tri Aj $abeq_tri4(i,j,n) => i<=j":

i<=j:12 states - 0ms
 (abeq_tri4(i,j,n))=>i<=j):4130 states - 60ms
  (A j (abeq_tri4(i,j,n))=>i<=j)):389 states - 108ms
Total computation time: 182ms.


### Apply the semigroup trick

In [39]:
%SGT comp_tri4 msd_tri Comp_tri4

Reading Result/comp_tri4.mpl
Rank: 85
392 states
values: [0, 1, 3, 5, 7, 9, 11, 13, 15, 17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 47, 49]
Writing Word Automata Library/Comp_tri4.txt
Done.

## 5-abelian complexity

Define the 5-block map of tri and construct the conversion predicate
between both numeration systems.

In [40]:
%%python
s5 = block(s, 5)
ns5 = address(s5, "trib5")
ns5.gen_ns()
(ns-ns5).gen_dfa("conv_tri_trib5")

>>> Address automaton for block(subst(0->01, 1->02, 2->0), 5)
  DT polynomial: X^3-X^2-X-1 (n=3)
  θ=1.839286755214161
  DT automaton: 11 states, 11 final states, 19 transitions.

>>> address(block(subst(0->01, 1->02, 2->0), 5), 'trib5') is generating it's NS
  Numeration system automaton: 3 states, 3 final states, 5 transitions.
  Writing /var/folders/l8/fphf93t95171xj2_rf0knxtc0000gn/T/tmp.wBJ3QLoyq3/Walnut/Custom Bases/msd_trib5.txt in format Walnut
  +address(block(subst(0->01, 1->02, 2->0), 5), 'trib5')+address(block(subst(0->01, 1->02, 2->0), 5), 'trib5')-address(block(subst(0->01, 1->02, 2->0), 5), 'trib5') is generating it's dfa under name msd_trib5_addition
  Combination polynomial is X^3-X^2-X-1
  θ=1.839286755214161
  Combination automaton: 1331 states, 1331 final states, 6859 transitions.
  (P,u) vector bound: (0.0, 0.0, 0.0) + (5.770539317125755, 8.360221985276333, 13.123391017013361)*C
  Fast poly bound: C in [(-1, -1, -1), (2, 2, 2)] => (34.62323590275453, 50.16133191165

Translate the border condition predicate into the current numeration
system.

In [41]:
def bordercond5 "?msd_trib5 (?msd_tri Eii,jj,kk,nn 
($conv_tri_trib5(?msd_tri ii, ?msd_trib5 i) & 
$conv_tri_trib5(?msd_tri jj, ?msd_trib5 j) & 
$conv_tri_trib5(?msd_tri kk, ?msd_trib5 k) & 
$conv_tri_trib5(?msd_tri nn, ?msd_trib5 n) & 
$bordercond(ii,jj,kk,nn)))":






(conv_tri_trib5(ii,i))&conv_tri_trib5(jj,j))):9 states - 0ms
 ((conv_tri_trib5(ii,i))&conv_tri_trib5(jj,j)))&conv_tri_trib5(kk,k))):27 states - 0ms
  (((conv_tri_trib5(ii,i))&conv_tri_trib5(jj,j)))&conv_tri_trib5(kk,k)))&conv_tri_trib5(nn,n))):81 states - 1ms
   ((((conv_tri_trib5(ii,i))&conv_tri_trib5(jj,j)))&conv_tri_trib5(kk,k)))&conv_tri_trib5(nn,n)))&bordercond(ii,jj,kk,nn))):148 states - 3ms
    (E ii , jj , kk , nn ((((conv_tri_trib5(ii,i))&conv_tri_trib5(jj,j)))&conv_tri_trib5(kk,k)))&conv_tri_trib5(nn,n)))&bordercond(ii,jj,kk,nn)))):148 states - 1ms
Total computation time: 5ms.


### Compute Parikh vectors

First we compute the Parikh vectors for the prefixes of Trib5.

In [42]:
%%python
for (i,a) in enumerate(ns5.alpha):
    w = {'_': 0}
    w[a] = 1
    parikh = address(s5, ns5.ns, **w)
    (parikh - ns5).gen_dfa(f"trib5p{i}")

>>> Address automaton for block(subst(0->01, 1->02, 2->0), 5)
  DT polynomial: X^5-X^4-X^3-X^2 (n=5)
  θ=1.839286755214161 (X^2)
  DT automaton: 11 states, 11 final states, 19 transitions.

>>> +address(block(subst(0->01, 1->02, 2->0), 5), 'trib5', '_': 0, 'a': 1)-address(block(subst(0->01, 1->02, 2->0), 5), 'trib5') is generating it's dfa under name trib5p0
  Combination polynomial is X^5-X^4-X^3-X^2
  θ=1.839286755214161 (X^2)
  Combination automaton: 121 states, 121 final states, 361 transitions.
  (P,u) vector bound: (0.0, 0.0, 0.0, 0.0, 0.0) + (13.995256614112588, 19.917967506314703, 30.811531205304703, 50.84791863393879, 87.70058065376499)*C
  Fast poly bound: C in [(-1, -2, -2, -1, -1), (0, 0, 1, 1, 1)] => (97.96679629878811, 139.4257725442029, 215.68071843713292, 355.93543043757154, 613.9040645763549)
Expanding DFA...done.
  Flattened automaton : 12655 states, 77 final states, 20673 transitions.
  Trimmed flattened automaton : 134 states, 77 final states, 232 transitions.
  Rea

### Compute the complexity function

Use Walnut to generate a linear representation for the 5-abelian
complexity.

In [43]:
def fac0 "?msd_trib5 Ex,y $trib5p0(i,x) & $trib5p0(i+n,y) & z+x=y":
def fac1 "?msd_trib5 Ex,y $trib5p1(i,x) & $trib5p1(i+n,y) & z+x=y":
def fac2 "?msd_trib5 Ex,y $trib5p2(i,x) & $trib5p2(i+n,y) & z+x=y":
def fac3 "?msd_trib5 Ex,y $trib5p3(i,x) & $trib5p3(i+n,y) & z+x=y":
def fac4 "?msd_trib5 Ex,y $trib5p4(i,x) & $trib5p4(i+n,y) & z+x=y":
def fac5 "?msd_trib5 Ex,y $trib5p5(i,x) & $trib5p5(i+n,y) & z+x=y":
def fac6 "?msd_trib5 Ex,y $trib5p6(i,x) & $trib5p6(i+n,y) & z+x=y":
def fac7 "?msd_trib5 Ex,y $trib5p7(i,x) & $trib5p7(i+n,y) & z+x=y":
def fac8 "?msd_trib5 Ex,y $trib5p8(i,x) & $trib5p8(i+n,y) & z+x=y":
def fac9 "?msd_trib5 Ex,y $trib5p9(i,x) & $trib5p9(i+n,y) & z+x=y":
def fac10 "?msd_trib5 Ex,y $trib5p10(i,x) & $trib5p10(i+n,y) & z+x=y":

(trib5p0(i,x))&trib5p0((i+n),y))):1385 states - 5ms
 (z+x)=y:149 states - 1ms
  ((trib5p0(i,x))&trib5p0((i+n),y)))&(z+x)=y):3569 states - 49ms
   (E x , y ((trib5p0(i,x))&trib5p0((i+n),y)))&(z+x)=y)):873 states - 6ms
Total computation time: 66ms.

(trib5p1(i,x))&trib5p1((i+n),y))):1265 states - 7ms
 (z+x)=y:149 states - 1ms
  ((trib5p1(i,x))&trib5p1((i+n),y)))&(z+x)=y):3294 states - 52ms
   (E x , y ((trib5p1(i,x))&trib5p1((i+n),y)))&(z+x)=y)):611 states - 6ms
Total computation time: 70ms.

(trib5p2(i,x))&trib5p2((i+n),y))):1183 states - 5ms
 (z+x)=y:149 states - 1ms
  ((trib5p2(i,x))&trib5p2((i+n),y)))&(z+x)=y):3105 states - 47ms
   (E x , y ((trib5p2(i,x))&trib5p2((i+n),y)))&(z+x)=y)):431 states - 6ms
Total computation time: 63ms.

(trib5p3(i,x))&trib5p3((i+n),y))):2537 states - 9ms
 (z+x)=y:149 states - 0ms
  ((trib5p3(i,x))&trib5p3((i+n),y)))&(z+x)=y):8085 states - 140ms
   (E x , y ((trib5p3(i,x))&trib5p3((i+n),y)))&(z+x)=y)):1353 states - 16ms
Total computation time: 174ms.

(tri

In [44]:
def min0 "?msd_trib5 Ei $fac0(i,n,x) & Aj,y $fac0(j,n,y) => y>=x":
def min1 "?msd_trib5 Ei $fac1(i,n,x) & Aj,y $fac1(j,n,y) => y>=x":
def min2 "?msd_trib5 Ei $fac2(i,n,x) & Aj,y $fac2(j,n,y) => y>=x":
def min3 "?msd_trib5 Ei $fac3(i,n,x) & Aj,y $fac3(j,n,y) => y>=x":
def min4 "?msd_trib5 Ei $fac4(i,n,x) & Aj,y $fac4(j,n,y) => y>=x":
def min5 "?msd_trib5 Ei $fac5(i,n,x) & Aj,y $fac5(j,n,y) => y>=x":
def min6 "?msd_trib5 Ei $fac6(i,n,x) & Aj,y $fac6(j,n,y) => y>=x":
def min7 "?msd_trib5 Ei $fac7(i,n,x) & Aj,y $fac7(j,n,y) => y>=x":
def min8 "?msd_trib5 Ei $fac8(i,n,x) & Aj,y $fac8(j,n,y) => y>=x":
def min9 "?msd_trib5 Ei $fac9(i,n,x) & Aj,y $fac9(j,n,y) => y>=x":
def min10 "?msd_trib5 Ei $fac10(i,n,x) & Aj,y $fac10(j,n,y) => y>=x":

y>=x:12 states - 0ms
 (fac0(j,n,y))=>y>=x):4486 states - 122ms
  (A j , y (fac0(j,n,y))=>y>=x)):97 states - 150ms
   (fac0(i,n,x))&(A j , y (fac0(j,n,y))=>y>=x))):2203 states - 6ms
    (E i (fac0(i,n,x))&(A j , y (fac0(j,n,y))=>y>=x)))):113 states - 4ms
Total computation time: 286ms.

y>=x:12 states - 0ms
 (fac1(j,n,y))=>y>=x):3341 states - 94ms
  (A j , y (fac1(j,n,y))=>y>=x)):97 states - 108ms
   (fac1(i,n,x))&(A j , y (fac1(j,n,y))=>y>=x))):1773 states - 5ms
    (E i (fac1(i,n,x))&(A j , y (fac1(j,n,y))=>y>=x)))):113 states - 2ms
Total computation time: 212ms.

y>=x:12 states - 0ms
 (fac2(j,n,y))=>y>=x):2568 states - 72ms
  (A j , y (fac2(j,n,y))=>y>=x)):97 states - 79ms
   (fac2(i,n,x))&(A j , y (fac2(j,n,y))=>y>=x))):1349 states - 3ms
    (E i (fac2(i,n,x))&(A j , y (fac2(j,n,y))=>y>=x)))):113 states - 3ms
Total computation time: 159ms.

y>=x:12 states - 0ms
 (fac3(j,n,y))=>y>=x):7320 states - 194ms
  (A j , y (fac3(j,n,y))=>y>=x)):120 states - 242ms
   (fac3(i,n,x))&(A j , y (fac

In [45]:
def diff0 "?msd_trib5 Ex,y $min0(n,x) & $fac0(i,n,y) & z+x=y":
def diff1 "?msd_trib5 Ex,y $min1(n,x) & $fac1(i,n,y) & z+x=y":
def diff2 "?msd_trib5 Ex,y $min2(n,x) & $fac2(i,n,y) & z+x=y":
def diff3 "?msd_trib5 Ex,y $min3(n,x) & $fac3(i,n,y) & z+x=y":
def diff4 "?msd_trib5 Ex,y $min4(n,x) & $fac4(i,n,y) & z+x=y":
def diff5 "?msd_trib5 Ex,y $min5(n,x) & $fac5(i,n,y) & z+x=y":
def diff6 "?msd_trib5 Ex,y $min6(n,x) & $fac6(i,n,y) & z+x=y":
def diff7 "?msd_trib5 Ex,y $min7(n,x) & $fac7(i,n,y) & z+x=y":
def diff8 "?msd_trib5 Ex,y $min8(n,x) & $fac8(i,n,y) & z+x=y":
def diff9 "?msd_trib5 Ex,y $min9(n,x) & $fac9(i,n,y) & z+x=y":
def diff10 "?msd_trib5 Ex,y $min10(n,x) & $fac10(i,n,y) & z+x=y":

(min0(n,x))&fac0(i,n,y))):3534 states - 16ms
 (z+x)=y:149 states - 1ms
  ((min0(n,x))&fac0(i,n,y)))&(z+x)=y):3346 states - 21ms
   (E x , y ((min0(n,x))&fac0(i,n,y)))&(z+x)=y)):2046 states - 15ms
Total computation time: 58ms.

(min1(n,x))&fac1(i,n,y))):2678 states - 17ms
 (z+x)=y:149 states - 2ms
  ((min1(n,x))&fac1(i,n,y)))&(z+x)=y):2555 states - 16ms
   (E x , y ((min1(n,x))&fac1(i,n,y)))&(z+x)=y)):1538 states - 12ms
Total computation time: 50ms.

(min2(n,x))&fac2(i,n,y))):2019 states - 10ms
 (z+x)=y:149 states - 1ms
  ((min2(n,x))&fac2(i,n,y)))&(z+x)=y):1999 states - 30ms
   (E x , y ((min2(n,x))&fac2(i,n,y)))&(z+x)=y)):1093 states - 9ms
Total computation time: 53ms.

(min3(n,x))&fac3(i,n,y))):4603 states - 24ms
 (z+x)=y:149 states - 1ms
  ((min3(n,x))&fac3(i,n,y)))&(z+x)=y):4301 states - 47ms
   (E x , y ((min3(n,x))&fac3(i,n,y)))&(z+x)=y)):2347 states - 17ms
Total computation time: 94ms.

(min4(n,x))&fac4(i,n,y))):2500 states - 12ms
 (z+x)=y:149 states - 1ms
  ((min4(n,x))&fac4(i,

In [46]:
def abeq_trib5 "?msd_trib5 $bordercond5(i,j,5,n+4) & (Ez $diff0(i,n,z) &  $diff0(j,n,z)) & (Ez $diff1(i,n,z) &  $diff1(j,n,z)) & (Ez $diff2(i,n,z) &  $diff2(j,n,z)) & (Ez $diff3(i,n,z) &  $diff3(j,n,z)) & (Ez $diff4(i,n,z) &  $diff4(j,n,z)) & (Ez $diff5(i,n,z) &  $diff5(j,n,z)) & (Ez $diff6(i,n,z) &  $diff6(j,n,z)) & (Ez $diff7(i,n,z) &  $diff7(j,n,z)) & (Ez $diff8(i,n,z) &  $diff8(j,n,z)) & (Ez $diff9(i,n,z) &  $diff9(j,n,z)) & (Ez $diff10(i,n,z) &  $diff10(j,n,z))":
def abeq_tri5 "?msd_tri (n<4 & $feq_tri(i,j,n)) | (n>=4 & (?msd_trib5 Eii,jj,nn ($conv_tri_trib5(?msd_tri i, ?msd_trib5 ii) & $conv_tri_trib5(?msd_tri j, ?msd_trib5 jj) & $conv_tri_trib5(?msd_tri n, ?msd_trib5 nn) & $abeq_trib5(ii,jj,nn-4))))":

computed ~:1 states - 1ms
computed ~:2 states - 0ms
(diff0(i,n,z))&diff0(j,n,z))):44965 states - 543ms
 (E z (diff0(i,n,z))&diff0(j,n,z)))):10392 states - 250ms
  (bordercond5(i,j,5,(n+4)))&(E z (diff0(i,n,z))&diff0(j,n,z))))):4228 states - 37ms
   (diff1(i,n,z))&diff1(j,n,z))):26522 states - 218ms
    (E z (diff1(i,n,z))&diff1(j,n,z)))):5140 states - 114ms
     ((bordercond5(i,j,5,(n+4)))&(E z (diff0(i,n,z))&diff0(j,n,z)))))&(E z (diff1(i,n,z))&diff1(j,n,z))))):3908 states - 22ms
      (diff2(i,n,z))&diff2(j,n,z))):17033 states - 85ms
       (E z (diff2(i,n,z))&diff2(j,n,z)))):2613 states - 49ms
        (((bordercond5(i,j,5,(n+4)))&(E z (diff0(i,n,z))&diff0(j,n,z)))))&(E z (diff1(i,n,z))&diff1(j,n,z)))))&(E z (diff2(i,n,z))&diff2(j,n,z))))):3578 states - 12ms
         (diff3(i,n,z))&diff3(j,n,z))):52636 states - 594ms
          (E z (diff3(i,n,z))&diff3(j,n,z)))):11689 states - 322ms
           ((((bordercond5(i,j,5,(n+4)))&(E z (diff0(i,n,z))&diff0(j,n,z)))))&(E z (diff1(i,n,z))&diff

In [47]:
eval comp_tri5 n "?msd_tri Aj $abeq_tri5(i,j,n) => i<=j":

i<=j:12 states - 0ms
 (abeq_tri5(i,j,n))=>i<=j):4247 states - 60ms
  (A j (abeq_tri5(i,j,n))=>i<=j)):398 states - 123ms
Total computation time: 197ms.


### Apply the semigroup trick

In [48]:
%SGT comp_tri5 msd_tri Comp_tri5

Reading Result/comp_tri5.mpl
Rank: 88
471 states
values: [0, 1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 61, 63]
Writing Word Automata Library/Comp_tri5.txt
Done.

## Check it out!

In [49]:
%%python
from itertools import product
from ratser.walimp import from_walnut

def valid(a, h, u):
    cur=a
    for c in u:
        v = h[cur]
        if c >= len(v):
            return False
        cur = v[c]
    return True

def enumrepr(a, h, k):
    alpha=list(range(max(map(len,h.values()))))
    for u in product(alpha, repeat=k):
        if valid(a, h, u):
            yield ''.join(map(str,u))

l = []
lbl = []
for k in range(1,6):
    lbl.append(str(k))
    with open(P(os.environ["WALNUT_HOME"]) / P(f"Result/comp_tri{k}.mpl")) as f:
            ser = from_walnut(f)
            l.append(ser)

(a,h)=s.subst()
print(f"{'n':>4}", *[f"{x+'-ab(tri)':>15}" for x in lbl])
print('-'*(4+16*len(lbl)))
for (i, u) in enumerate(enumrepr(a, h, 8)):
    print(f"{i:4}", *[ f"{int(s.value(u)):15}" for s in l ])

   n       1-ab(tri)       2-ab(tri)       3-ab(tri)       4-ab(tri)       5-ab(tri)
------------------------------------------------------------------------------------
   0               1               1               1               1               1
   1               3               3               3               3               3
   2               3               5               5               5               5
   3               4               7               7               7               7
   4               3               8               9               9               9
   5               4              10              11              11              11
   6               4              10              13              13              13
   7               4              10              14              15              15
   8               3               9              14              17              17
   9               4              10              16             

## Let’s wrap it up!

In [50]:
%%shell
cd $WALNUT_HOME
cat << EOF | tr '\n' '\0' | tar cvzf /tmp/abelian.tar.gz --null -T -
Custom Bases/msd_tri.txt
Custom Bases/msd_tri_addition.txt
Word Automata Library/Tri.txt
Automata Library/feq_tri.txt
Result/comp_tri.mpl
Word Automata Library/Comp_tri1.txt
Result/comp_tri1.mpl
Automata Library/abeq_tri1.txt
Word Automata Library/Comp_tri2.txt
Result/comp_tri2.mpl
Automata Library/abeq_tri2.txt
Word Automata Library/Comp_tri3.txt
Result/comp_tri3.mpl
Automata Library/abeq_tri3.txt
Word Automata Library/Comp_tri4.txt
Result/comp_tri4.mpl
Automata Library/abeq_tri4.txt
Word Automata Library/Comp_tri5.txt
Result/comp_tri5.mpl
Automata Library/abeq_tri5.txt
EOF

a Custom Bases/msd_tri.txt
a Custom Bases/msd_tri_addition.txt
a Word Automata Library/Tri.txt
a Automata Library/feq_tri.txt
a Result/comp_tri.mpl
a Word Automata Library/Comp_tri1.txt
a Result/comp_tri1.mpl
a Automata Library/abeq_tri1.txt
a Word Automata Library/Comp_tri2.txt
a Result/comp_tri2.mpl
a Automata Library/abeq_tri2.txt
a Word Automata Library/Comp_tri3.txt
a Result/comp_tri3.mpl
a Automata Library/abeq_tri3.txt
a Word Automata Library/Comp_tri4.txt
a Result/comp_tri4.mpl
a Automata Library/abeq_tri4.txt
a Word Automata Library/Comp_tri5.txt
a Result/comp_tri5.mpl
a Automata Library/abeq_tri5.txt
